In [56]:
import requests
from typing import Dict, Any
def chat_with_model(token: str, question: str) -> Dict[str, Any]:
    """
    Send a question to the chat model and return the response.
    
    Args:
        token: API token for authentication
        question: The question to ask the model
        
    Returns:
        dict: The JSON response from the API
    """
    url = 'http://localhost:3000/api/chat/completions'
    headers = {
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json'
    }
    data = {
        "model": "gemma3:27b",
        "messages": [
            {
                "role": "user",
                "content": question,
                "system": ""
            }
        ],
        "files": [{"type": "collection", "id": "a0875df9-e60e-49b2-b6da-e256dca2d4f6"}]
    }

    #print ("Query: ")
    #print (data)
    
    response = requests.post(url, headers=headers, json=data, timeout=60)
    response.raise_for_status()
    return response.json()["choices"][0]["message"]["content"]

In [57]:
token = "sk-317f069c260446b5bf3fb8c4beb91e80"

In [58]:
result = chat_with_model(token, "Seit wann ist es in Deutschland möglich, zwei Fahrzeuge abwechselnd mit ein und demselben Nummernschild zu fahren?")  # Example question

print(result)



Seit dem 01.07.2012 ist es in Deutschland möglich, zwei Fahrzeuge abwechselnd mit ein und demselben Nummernschild zu fahren [2]. Beide Fahrzeuge müssen jedoch der gleichen Fahrzeugklasse angehören und die gleiche Kennzeichengröße haben [2].


In [59]:
# load qa pair from question file
# for each question, call chat_with_model and map to SingleTurnSample
import json
from pathlib import Path
from typing import List
from tqdm import tqdm
from ragas.dataset_schema import SingleTurnSample

# Load question-answer pairs from JSON file
qa_file = Path("question_answer_pairs.json")
with open(qa_file, 'r', encoding='utf-8') as f:
    qa_pairs = json.load(f)

print(f"Loaded {len(qa_pairs)} question-answer pairs")

# Create SingleTurnSample objects
samples: List[SingleTurnSample] = []

# Process each Q&A pair
for qa in tqdm(qa_pairs, desc="Processing Q&A pairs"):  # Start with first 5 for testing
    try:
        # Get response from the model
        model_response = chat_with_model(token, qa["question"])
        
        # Create SingleTurnSample
        sample = SingleTurnSample(
            user_input=qa["question"],
            response=model_response,
            reference=qa["answer"]
        )
        samples.append(sample)
        
    except Exception as e:
        print(f"Error processing question: {qa['question'][:50]}... - {e}")

print(f"\nCreated {len(samples)} SingleTurnSample objects")

Loaded 195 question-answer pairs


Processing Q&A pairs: 100%|██████████| 195/195 [21:38<00:00,  6.66s/it]


Created 195 SingleTurnSample objects


In [60]:

## pretty print the first sample
from pprint import pprint
pprint(samples[0].to_dict())  # Assuming SingleTurnSample has a to_dict method


{'reference': 'Für Urkunden des Sächsischen Staatsministeriums der Justiz ist '
              'das Sächsisches Staatsministerium der Justiz zuständig. Für '
              'Urkunden der Landgerichte und ihrer Gerichte, sowie alle '
              'anderen Gerichte und Notare im jeweiligen Landgerichtsbezirk '
              'sind die Präsidenten der Landgerichte zuständig.',
 'response': 'Wenn eine Urkunde von Behörden des Freistaates Sachsen aus dem '
             'Bereich der Justiz ausgestellt wurde, ist das Sächsische '
             'Staatsministerium der Justiz [1] oder die Präsidenten der '
             'Landgerichte [1] zuständig für die Erteilung einer Apostille.',
 'user_input': 'Welche Behörde ist für die Erteilung einer Apostille '
               'zuständig, wenn eine Urkunde von Behörden des Freistaates '
               'Sachsen aus dem Bereich der Justiz ausgestellt wurde?'}


In [ ]:
from langchain_openai import ChatOpenAI
from ragas.llms import LangchainLLMWrapper

    # Adjust the URL as needed
    
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gemma3:27b", base_url='http://185.128.119.215:11434/v1', api_key="ollama" ))

In [62]:
from ragas.metrics import AnswerAccuracy
scorer = AnswerAccuracy(llm=evaluator_llm) # evaluator_llm wrapped with ragas LLM Wrapper

"""
for sample in tqdm(samples, desc="Scoring samples"):
    try:
        # Score each sample
        score = await scorer.single_turn_ascore(sample)
        #sample.score = score  # Assuming SingleTurnSample has a score attribute
        pprint(sample.to_dict())
        print(score)
        print("---")
        
    except Exception as e:
        print(f"Error scoring sample: {sample.user_input[:50]}... - {e}")
#score = await scorer.single_turn_ascore(sample)
"""

'\nfor sample in tqdm(samples, desc="Scoring samples"):\n    try:\n        # Score each sample\n        score = await scorer.single_turn_ascore(sample)\n        #sample.score = score  # Assuming SingleTurnSample has a score attribute\n        pprint(sample.to_dict())\n        print(score)\n        print("---")\n\n    except Exception as e:\n        print(f"Error scoring sample: {sample.user_input[:50]}... - {e}")\n#score = await scorer.single_turn_ascore(sample)\n'

In [63]:
from ragas import evaluate
from ragas import SingleTurnSample, EvaluationDataset

dataset = EvaluationDataset(samples=samples)

results = evaluate(dataset, metrics=[scorer])
results

Evaluating: 100%|██████████| 195/195 [03:17<00:00,  1.01s/it]


{'nv_accuracy': 0.6090}

In [75]:
df = results.to_pandas()

df

,user_input,response,reference,nv_accuracy
0,Welche Behörde ist für die Erteilung einer Apo...,Wenn eine Urkunde von Behörden des Freistaates...,Für Urkunden des Sächsischen Staatsministerium...,0.75
1,Unter welchen Rufnummern stehen Beraterinnen u...,Beraterinnen und Berater stehen in Sachsen run...,Sie stehen unter den einheitlichen Rufnummern ...,0.25
2,Welche Gewerbe müssen in Deutschland eine beso...,"Gewerbe, die nicht zu den erlaubnisfreien Gewe...",Ausnahme von den erlaubnisfreien Gewerben bild...,0.50
3,Welche Personenkreise sind an die Betreuungsve...,An die Betreuungsverfügung sind sowohl das zus...,Sowohl der von Ihnen benannte Betreuer als auc...,0.75
4,Wann sollten Sie sich bei der Bundesagentur fü...,"Sobald absehbar ist, dass Sie nach der Ausbild...","Sobald absehbar ist, dass Sie nach der Ausbild...",1.00
...,...,...,...,...
190,Welche Institutionen oder Personengruppen werd...,Im Insolvenzverfahren werden die Interessen de...,Die Interessen der Gläubiger werden im Insolve...,1.00
191,Gemäß welchem Paragraphen des Sozialgesetzbuch...,"I am sorry, but the provided context does not ...",Die Betreuung und Unterstützung wird als erzie...,0.00
192,Was sind die Verpflichtungen des Käufers und d...,"Beim Kaufvertrag verpflichtet sich der Käufer,...",Beim Kaufvertrag verpflichtet sich der Käufer ...,1.00
193,Welche Formate können Marken haben?,Marken können folgende Formate haben: Wortmark...,"Marken können folgende Form haben: Wortmarken,...",1.00


In [79]:
# filter out samples with score < 0.5
filtered_df = df[df['nv_accuracy'] <= 0.5]
filtered_df.head()

# Save the filtered results to a CSV file
filtered_df.to_html("filtered_results.html", index=False)